## Visualizing Results of the Deletion Capacity Experiment

These are the results of the deletion capacity experiment. 

At a high level, we're seeing very conservative regret bounds for the Memory Pair. This means that we're requiring large sample complexity in return for a very low deletion capacity.

It's also worth noting that our sample complexity (bar for a good learner) increases as the data wiggles more. When the Lipschitz constant and upper-bound on the Hessian are high, the sample complexity jumps and the amount of noise injected to the model becomes destabilizingly high.

Goals:
- Analyze the simulation results from the experiment runs and visualize the cumulative regret
- Focus on $\widehat{G}$ such that we can see its impact on the downstream stability of the learner
- Investigate alternative methods of privacy accounting. Can we get tigheter regret bounds such that we don't inject so much noise into the parameter estimates.


### Page-Wide Questions
- The formulas for sample complexity and deletion capacity look very similar (ie. use the $GD$ term). Why is this the case, and what does this suggest about the relationship between these two formulas? If I were to divide sample complexity by deletion capacity, it would almost look like something like a harmonic mean.
- I wonder how $\widehat{D}$ is being estimated. It looks like a lot of seeds are capping it at 10, which is a worst-case scenario. Is there something that can reduce this?

In [1]:
import pandas as pd
import numpy as np
import random
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# crawl through the directory
data = pd.DataFrame()
for root, dirs, files in os.walk("grid_2025_01_01/sweep/split_0.5-0.5_q0.90_k10_rdp_eps1.0"):
    for filename in files:
        if filename.endswith(".csv"):
            df = pd.read_csv(os.path.join(root, filename))
            data = pd.concat([data, df], ignore_index=True)

The code performs a grid search over the experiment parameters. For each seed,
1. **(Calibration.)** a `Calibrator` object draws a small sample of the data stream to estimate stream-attributes like the Lipschitz constant $L$, the upper and lower bound of the Hessian eigenvalues $C, C$, and the resulting sample complexity required to meet predefined accuracy goals.
2. **(Warmup.)** the model is trained on a stream of samples until it reaches sample complexity. This sets the model up for success when we test deletions.
3. **(Workload.)** a stream of interleaved insertions and deletions is passed to the model. It's expected to service the requests in the order they're given.

In [3]:
summary_statistics = data.describe()
summary_statistics.columns

Index(['C_hat', 'D_hat', 'G_hat', 'N_star_theory', 'acc', 'c_hat',
       'delta_total', 'eps_converted', 'eps_remaining', 'event', 'm_theory',
       'regret', 'sigma_step_theory', 'seed', 'avg_regret_empirical',
       'N_star_emp', 'm_emp', 'final_acc', 'total_events', 'gamma_learn',
       'gamma_priv', 'quantile', 'delete_ratio', 'eps_total'],
      dtype='object')

In [4]:
# get the individual event types
print(data["event_type"].unique())
print(data["event_type"].value_counts())

['calibrate' 'warmup' nan 'insert' 'delete']
event_type
warmup       399842
calibrate      2500
insert           20
delete            1
Name: count, dtype: int64


In [5]:
data

,C_hat,D_hat,G_hat,N_star_theory,acc,c_hat,delta_total,eps_converted,eps_remaining,event,...,N_star_emp,m_emp,final_acc,total_events,gamma_learn,gamma_priv,quantile,delete_ratio,accountant,eps_total
0,NaN,NaN,NaN,NaN,4.665831,NaN,0.00001,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,4.199533,NaN,0.00001,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,26.992378,NaN,0.00001,0.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,32.809801,NaN,0.00001,0.0,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,7.622270,NaN,0.00001,0.0,1.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402363,1.0,4.781109,4.724902,2042.0,535.829887,1.0,0.00001,NaN,NaN,2608.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
402364,1.0,4.781109,4.724902,2042.0,97.990825,1.0,0.00001,NaN,NaN,2609.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
402365,1.0,4.781109,4.724902,2042.0,1099.841942,1.0,0.00001,NaN,NaN,2610.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
402366,1.0,4.781109,4.724902,2042.0,792.711441,1.0,0.00001,NaN,NaN,2611.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
seed_level_data = data.loc[data["event_type"].isnull()]
event_level_data = data.loc[~data["event_type"].isnull()]

In [7]:
event_level_data

,C_hat,D_hat,G_hat,N_star_theory,acc,c_hat,delta_total,eps_converted,eps_remaining,event,...,N_star_emp,m_emp,final_acc,total_events,gamma_learn,gamma_priv,quantile,delete_ratio,accountant,eps_total
0,NaN,NaN,NaN,NaN,4.665831,NaN,0.00001,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,4.199533,NaN,0.00001,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,26.992378,NaN,0.00001,0.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,32.809801,NaN,0.00001,0.0,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,7.622270,NaN,0.00001,0.0,1.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402363,1.0,4.781109,4.724902,2042.0,535.829887,1.0,0.00001,NaN,NaN,2608.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
402364,1.0,4.781109,4.724902,2042.0,97.990825,1.0,0.00001,NaN,NaN,2609.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
402365,1.0,4.781109,4.724902,2042.0,1099.841942,1.0,0.00001,NaN,NaN,2610.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
402366,1.0,4.781109,4.724902,2042.0,792.711441,1.0,0.00001,NaN,NaN,2611.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# check event level data for theoretical deletion capacity
seed_level_data["m_emp"].value_counts()

m_emp
0.0    4
1.0    1
Name: count, dtype: int64

### What does this data mean?

The data we get from the experiment is incredibly granular. This is good because we can isolate the impact of different operations on the regret. A list of the parameters is included below:

`Data Stream Attributes`
- $q$ is the quantile used for selecting the parameter estimates so we don't accidentally pull a high-ass parameter estimate
- $\widehat{C}$ is the upper bound on the Hessian eigenvalues
- $\widehat{C}$ is the lower bound on the Hessian eigenvalues
- $\widehat{D}$ is the upper bound of the diameter of the ellipsoid
- $\widehat{G}$ is the Lipschitz constant of the function, representing how much the output of the function changes as the inputs change
- $N^{\star}_{theory}$ is the theoretical sample complexity to reach the specified amount of average-regret

`Workload Parameters`
- $k$ is the number of insertions per delete operation
- $m_{emp}$ is the empirical deletion capacity of the seed

`Privacy Parameters`
- $\delta_{total}$ and $\varepsilon_{total}$ are the total $(\varepsilon,\delta)$ budget given the the accountant
- $\delta_{step}$ and $\varepsilon_{step}$ are the amount of privacy "spent" per deletion


`Event-Level Attributes`
- $event$ is the zero-based index of the operation within the seed run
- $avg\_regret\_empirical$ is the mean per-operation regret for the stream of events up to this point

## Theoretical Sample Complexities

We can calculate theoretical sample complexities using the data we get from calibration. 

The formula for the sample complexity is based entirely on the attributes of our data stream and its spread: $G$, $D$, and $\sqrt{cC}$ and so the estimates from our calibration period actually mean a lot. A large estimate for Lipschitz constant, or the bounds of our Hessian eigenvalues means we'll have an artificially inflated Sample Complexity.

$$
S = [\frac{GD\sqrt{Cc}}{\gamma_{learn}}]^{2}
$$

It's also worth noting that the sample complexity is already quite conservative because of the method used for accouting. 

In [9]:
sample_complexity_calculations = seed_level_data[["seed", "N_star_theory", "C_hat", "c_hat", "D_hat", "G_hat", "gamma_learn"]]
sample_complexity_calculations

,seed,N_star_theory,C_hat,c_hat,D_hat,G_hat,gamma_learn
99950,2.0,106043.0,1.0,1.0,10.000000,16.282052,0.5
99951,4.0,163128.0,1.0,1.0,10.000000,20.194495,0.5
299852,3.0,460722.0,1.0,1.0,10.000000,33.938231,0.5
399803,0.0,2042.0,1.0,1.0,4.781109,4.724902,0.5
399804,1.0,662801.0,1.0,1.0,10.000000,40.706270,0.5


### Interpreting $\gamma$ Parameters

**Question:** What is the interpretation of $\gamma_{learn}$ and how is it used to calculate sample complexity and deletion capacity?

**Answer:** If $\gamma_{learn}$ is the amount of slack given to the learner, then a $\gamma_{learn}$ of `0.5` is really inflating my sample complexity by 4. Consider a larger $\gamma_{learn}$ for the first round of experiments so that you don't blow up your sample complexity too early.

The large sample complexities can also be an issue because our `max_events` parameter is set to 100000. So if the sample complexity is any larger than that, then the learner wouldn't even be able to unlearn a single point.

**Question:** Okay, so we have two parameters $\gamma_{learn}$ and $\gamma_{private}$, why do we need them both? What's the difference between the learning parameter or the private parameter?

**Answer:** They were separated because we need two separate slack parameters. One is used to bound the average regret during the learning period, and the second is used to bound the average regret when processing the workload.

### Effects of Limited Convexity

If the loss function is only weakly convex, then the experiment would end before the sample complexity is reached, and so even doing a single insertion would be a waste of time. I'm increasing the maximum number of events to allow for more of the experiments to reach this stage.

**Note:** a suggestion would be to replace the two gamma parameters with a single $\alpha$ that's used to split the amount of slack given to deletions versus insertions. 


## Theoretical Deletion Capacities 

The $\gamma_{priv}$ is also used to calculate deletion capacity. The quantifies the amount of cumulative regret you're willing to pay for all future deletions. It's used to calculate the upper bound on deletion capacity.

$$
m \leq \gamma_{priv} \times \frac{N^{\star}}{GD + \sigma\sqrt{2N^{*}\ln{\frac{1}{\delta_{step}}}}}
$$

The deletion capacity is only determined once the warmup has completed. We use the calibration statistics and the results from the warmup to calculate the theoretical deletion capacity for the experiment. This is the maximum number of deletions served (although many seeds never reach that point) and is used to calibrate the noise in the standard odometer.

For some reason, we're not getting the $m_{theory}$ that we need to actually run the experiment.

In [10]:
deletion_capacity_data = seed_level_data[["seed", "m_theory","m_emp", "gamma_priv", "G_hat", "D_hat", "N_star_theory", "sigma_step_theory"]]
deletion_capacity_data

,seed,m_theory,m_emp,gamma_priv,G_hat,D_hat,N_star_theory,sigma_step_theory
99950,2.0,NaN,0.0,0.5,16.282052,10.000000,106043.0,295.657535
99951,4.0,NaN,0.0,0.5,20.194495,10.000000,163128.0,295.657535
299852,3.0,NaN,0.0,0.5,33.938231,10.000000,460722.0,295.657535
399803,0.0,1.0,1.0,0.5,4.724902,4.781109,2042.0,295.657535
399804,1.0,NaN,0.0,0.5,40.706270,10.000000,662801.0,295.657535
